# Explore the Externally-Cued Movements dataset

## Data access

This notebook demos selected content and datatypes of this dataset. Data are available in the form of a
[DataLad](https://datalad.org) dataset. This is a portable, versioned representation of the data that provides access to data and metadata. To learn more about working with DataLad datasets, visit the [DataLad handbook](https://handbook.datalad.org).

The next cell obtains the dataset, and downloads the subset of the contained data relevant for the demos shown on this page. However, the dataset links all data, and additional downloads can be made, following the same approach, to explore more dataset facets.

In [ ]:
# obtain the DataLad dataset, and place it into the 'data' subdirectory
!datalad clone https://github.com/sfb1451/B03_externally-cued-movements.git data
# use DataLad to download files matching subject 001 in the 'data' directory
!datalad -C data get sub-001

## Data demos

The following steps demo a range of data properties using a single subject's data files (sub-001).

The next cell configures the computational environment for the demo code below. The demos use the [MNE](https://mne.tools) software package. All code assumes that it is executed in the `eeg` directory of a particular subject.


In [ ]:
# install MNE
!pip install -q mne

### Step 1: Load the EEGLAB file

First, load the EEGLAB .set file. This file contains epoched data.

In [ ]:
# start from here after restarting the notebook kernel
%matplotlib notebook

# enter the data directory
%cd data/sub-001/eeg

In [ ]:
import pandas as pd
import mne
from mne.stats import permutation_t_test

electrodes = pd.read_csv('sub-001_task-extstim_electrodes.tsv', delimiter='\t')
events = pd.read_csv('sub-001_task-extstim_events.tsv', delimiter='\t')

# Replace 'path_to_your_file.set' with the path to your .set file
set_file_path = 'sub-001_task-extstim_eeg.set'
epochs = mne.io.read_epochs_eeglab(set_file_path)

### Step 2: Basic Plot of Epochs

Start by plotting the epochs to get a basic understanding of the data.

In [ ]:
epochs.plot(n_epochs=10, picks=['eeg'], title='Sample Epochs')


### Step 3: Plot ERP (Event-Related Potentials) Images

ERP images plot the data of each epoch as a single line in an image, which is a good way to visualize condition-specific responses.

In [ ]:
# Specify the event or condition you're interested in
event_id = 'S 104'  # Replace with your specific event ID

# Plot ERP image for a specific channel
epochs[event_id].plot_image(picks=['C3'], sigma=1.0, cmap='viridis')


### Step 4: Topographic Maps

Topographic maps show the potential field on the scalp, which can be useful to visualize the distribution of brain activity at specific times.

In [ ]:
import numpy as np

# Time points in seconds (e.g., 250 ms post-stimulus)
times = [0.25]
epochs[event_id].average().plot_topomap(times=times, ch_type='eeg')

# pick EEG Channels
picks = mne.pick_types(
    epochs.info, meg=False, eeg=True, stim=False, eog=True, exclude="bads"
)
data = epochs.get_data()
times = epochs.times

temporal_mask = np.logical_and(0.2 <= times, times <= 0.3)
data = np.mean(data[:, :, temporal_mask], axis=2)

n_permutations = 50000
T0, p_values, H0 = permutation_t_test(data, n_permutations, n_jobs=None)

significant_sensors = picks[p_values <= 0.05]
significant_sensors_names = [epochs.ch_names[k] for k in significant_sensors]

print("Number of significant sensors : %d" % len(significant_sensors))
print("Sensors names : %s" % significant_sensors_names)

evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis], epochs.info, tmin=0.25)

# Extract mask and indices of active sensors in the layout
mask = p_values[:, np.newaxis] <= 0.05

evoked.plot_topomap(
    ch_type="eeg",
    times=[0.25],
    scalings=1,
    time_format=None,
    cmap="Reds",
    vlim=(0.0, np.max),
    units="-log10(p)",
    cbar_fmt="-%0.1f",
    mask=mask,
    size=3,
    show_names=lambda x: x[4:] + " " * 20,
    time_unit="s",
)

### Step 5: Time-Frequency Representation

Time-frequency plots can show how the frequency content of the signal changes over time, which is useful for analyzing oscillatory activity.

In [ ]:
from mne.time_frequency import tfr_morlet
import numpy as np

# Choose channels and frequencies
channels = ['C4']
freqs = np.arange(2, 48)  # 2 to 48 Hz
n_cycles = freqs / 2.  # Different number of cycle per frequency

power, itc = tfr_morlet(epochs[event_id], freqs=freqs, n_cycles=n_cycles, return_itc=True, picks=channels)

# Plotting the time-frequency power
power.plot(baseline=(-1.5, -0.5), mode='logratio', title='Time-Frequency Amplitude', vmin=-0.5, vmax=0.5)

# Plotting the phase-locking value (ITC) with the color scale from 0 to 1
itc.plot(title='Phase-locking', vmin=0, vmax=1)

